In [3]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import os
import requests
import time
from random import uniform
import pandas as pd

In [16]:
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)
#여성 상의
#driver.get('https://shopping.naver.com/style/style/category?menu=10000532')
# 여성 하의 스커트
driver.get('https://shopping.naver.com/style/style/category?menu=10000548')
#남성
#driver.get('https://shopping.naver.com/style/style/category?menu=10000623')

In [17]:
# 스크롤
link_list = []

for scr in range(10):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select('div._1aNmedx3I0 > ul._3KH4iCkMJ_ > li > div > a')
    f = uniform(2.0, 2.5)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(f)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(f)
    
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    for link in links:
        link_list.append('https://shopping.naver.com'+link['href'])
        
    
    if new_height == last_height:

            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(f)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue


In [21]:
link_list

['https://shopping.naver.com/style/style/stores/100070822/products/4860657503',
 'https://shopping.naver.com/style/style/stores/100614424/products/4860394555',
 'https://shopping.naver.com/style/style/stores/100546823/products/4861250667',
 'https://shopping.naver.com/style/style/stores/100435958/products/4860455273',
 'https://shopping.naver.com/style/style/stores/100081570/products/4860442073',
 'https://shopping.naver.com/style/style/stores/100576863/products/4861140421',
 'https://shopping.naver.com/style/style/stores/100148561/products/4829503134#INTRODUCE',
 'https://shopping.naver.com/style/style/stores/1000024326/products/4861164903',
 'https://shopping.naver.com/style/style/stores/100013780/products/4860964180',
 'https://shopping.naver.com/style/style/stores/100067106/products/4860735715',
 'https://shopping.naver.com/style/style/stores/1000005290/products/4860817808',
 'https://shopping.naver.com/style/style/stores/1000016577/products/4860575101',
 'https://shopping.naver.co

In [22]:
link_list = list(set(link_list))

In [23]:
# csv 파일로 저장
import pandas as pd
results = pd.DataFrame(link_list, columns=['link_addr'])
results.to_csv('./link_addr_skirt4.csv', header=False) #, mode='a', header=False)

In [24]:
#csv 파일 가져오기
import csv
f = open('./link_addr_skirt4.csv','rt')
link_addr_file = csv.reader(f)

In [25]:
link_list = []
for row in link_addr_file :
    link_list.append(row[1])

In [20]:
#상세 페이지 들어가서 원하는 정보 저장
i = 0

while i < len(link_list):
    text_list = []
    
    driver.get(link_list[i])
    print(link_list[i])
    
    f = uniform(1.5, 1.7)
    time.sleep(f)
    
    a = ""
    a = link_list[i].split("/products/")
    a[1][:10]
    
    i += 1
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        images = soup.find('div','_396uGDX0Fi')
        # soup.select -> soup.find
        # ('div._3utq7rRVBL > div.editor_wrap > div > div > div.se_component_wrap > div.se_component > 
        # div.se_sectionArea > div.se_editArea > div.se_viewArea > a > img')
        name = soup.find('h3','_2IA5sp7BRM').get_text() # 동일
        price = soup.find('strong','WvptIJtvXo').get_text()  
        tag=soup.find('div','_24FKuaBdoK').get_text() # 동일
        info = [soup.find_all('th','_15qeGNn6Dt')[n].string for n in range(0,20)]  # 동일
        detail = [soup.find_all('td','jvlKiI0U_y')[n].string for n in range(0,20)] # 동일
        texts = soup.find_all('p', 'se_textarea') #동일


        if not os.path.isdir(a[1][:10]):
            os.mkdir(a[1][:10])     

        for text in texts:
            text_list.append(text.get_text().strip().replace(u'\xa0', u' '))
            text_list2 = [zz for zz in text_list if zz]

        # 긁어올 카테고리 명을 입력하세요     
        data_1 = ({'category':'여성/하의/스커트', 'name': [name], 'price': [price] , 'tag':[tag], 'size':[text_list2]})   #, 'price': [price]
        df1 = pd.DataFrame(data_1)

        data_2 = [detail]
        df2 = pd.DataFrame(data_2, columns = info)

        df3 = pd.concat([df1,df2], axis=1)

        df3.to_csv(".\\"+a[1][:10]+"\\csv_"+a[1][:10]+".csv", index = True, encoding='utf-8')

        for image in images:

            src_page = requests.get(image['src'], headers={"Referer":"https://www.naver.com/"})
            src_image = src_page.content
            with open(".\\"+a[1][:10]+"\\img_"+a[1][:10]+'_'+str(images.index(image))+".png", "wb") as downfile:
                downfile.write(src_image)
            f = uniform(0.4, 0.6)    
            time.sleep(f)

    except:
        print("상품정보 가져오기 오류발생")
        pass

https://shopping.naver.com/style/style/stores/100070822/products/4860657503
https://shopping.naver.com/style/style/stores/100614424/products/4860394555
https://shopping.naver.com/style/style/stores/100546823/products/4861250667
https://shopping.naver.com/style/style/stores/100435958/products/4860455273
https://shopping.naver.com/style/style/stores/100081570/products/4860442073
https://shopping.naver.com/style/style/stores/100576863/products/4861140421
https://shopping.naver.com/style/style/stores/100148561/products/4829503134#INTRODUCE
https://shopping.naver.com/style/style/stores/1000024326/products/4861164903
https://shopping.naver.com/style/style/stores/100013780/products/4860964180
https://shopping.naver.com/style/style/stores/100067106/products/4860735715
https://shopping.naver.com/style/style/stores/1000005290/products/4860817808
https://shopping.naver.com/style/style/stores/1000016577/products/4860575101
https://shopping.naver.com/style/style/stores/100390743/products/4860438874

https://shopping.naver.com/style/style/stores/1000007213/products/4861273169
https://shopping.naver.com/style/style/stores/100093113/products/4860467784
https://shopping.naver.com/style/style/stores/100628644/products/4861324699
https://shopping.naver.com/style/style/stores/100502847/products/4861242372
https://shopping.naver.com/style/style/stores/100238067/products/4860831294
https://shopping.naver.com/style/style/stores/100576409/products/4860650083
https://shopping.naver.com/style/style/stores/100357868/products/4860562761
https://shopping.naver.com/style/style/stores/100090278/products/4860680109
https://shopping.naver.com/style/style/stores/1000007227/products/4860595650
https://shopping.naver.com/style/style/stores/100148561/products/4857811391#INTRODUCE
https://shopping.naver.com/style/style/stores/100203970/products/4860459827
https://shopping.naver.com/style/style/stores/100653347/products/4860186368#INTRODUCE
https://shopping.naver.com/style/style/stores/100117799/products/4

https://shopping.naver.com/style/style/stores/1000002596/products/4860603715
https://shopping.naver.com/style/style/stores/1000003910/products/4860485383
https://shopping.naver.com/style/style/stores/100640211/products/4861308440
https://shopping.naver.com/style/style/stores/1000014248/products/4861175790
https://shopping.naver.com/style/style/stores/1000013533/products/4860636436
https://shopping.naver.com/style/style/stores/100191665/products/4860727239
https://shopping.naver.com/style/style/stores/100630521/products/4848652889#INTRODUCE
https://shopping.naver.com/style/style/stores/100630521/products/4811807147#INTRODUCE
https://shopping.naver.com/style/style/stores/1000007213/products/4860475635
https://shopping.naver.com/style/style/stores/1000014415/products/4860456713
https://shopping.naver.com/style/style/stores/100093803/products/4861316236
https://shopping.naver.com/style/style/stores/100013780/products/4861088307
https://shopping.naver.com/style/style/stores/100374444/produc

In [6]:
import numpy as np
import cv2 as cv
import cv2
import glob
import os
from matplotlib import pyplot as plt
import cv2
from matplotlib import pyplot as plt

In [26]:
path = "c:/py_data/final_project" #이미지 있는 경로 입력 
file_list = os.listdir(path)
file_list_png = [file for file in file_list if file.endswith(".png")] #png 파일만 불러오기


save_path = "./running_data" #저장할 폴더명을 입력하세요(폴더 자동생성) 
save_path2 = save_path + "/"
if not os.path.isdir(save_path):
    os.mkdir(save_path)


for file_list in file_list_png:
    
    image_path = path + "\\" + file_list
    img = cv2.imread(image_path)
    
    dst = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)    
    cv2.imwrite(save_path2 + file_list+".png",dst)

In [27]:
images = glob.glob("./test2/*.png")

for i in images:
    a = i.split("\\")
    print(a[1][:17])

In [28]:
path = "./test2"
image_path = path+"\\"+image

file_list = os.listdir(path)
file_list_png = [file for file in file_list if file.endswith(".png")]

print ("file_list_py: {}".format(file_list_py))

images = glob.glob("./test2/*.png")
for image in file_list_png:
    
    image_path = path + "\\" + image
    print(image_path)

TypeError: must be str, not Tag

In [ ]:
# 드라이버 종료
driver.close()